# Test HIM and KLqp in Edward

- 2018.1.23

### Dataset iris with features [-1,1] normalized.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal
from tensorflow.contrib import slim

from sklearn.datasets import load_iris
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [2]:
def normalizer(array):
    """normalize the array(axis=0) to [head, tail]"""
    amin, amax = array.min(axis=0), array.max(axis=0)
    return ((np.subtract(array, amin) / (amax - amin))*2-1.)

def generator(arrays, batch_size):
    """Generate batches, one with respect to each array's first axis."""
    starts = [0] * len(arrays)    # pointers to where we are in iteration
    while True:
        batches = []
        for i, array in enumerate(arrays):
            start = starts[i]
            stop = start + batch_size
            diff = stop - array.shape[0]
            if diff <= 0:
                batch = array[start:stop]
                starts[i] += batch_size
            else:
                batch = np.concatenate((array[start:], array[:diff]))
                starts[i] = diff
            batches.append(batch)
        yield batches

In [3]:
ed.set_seed(42)

In [4]:
M = 20    # batch size during training
D = 4    # number of features

In [5]:
iris = load_iris()
Value = normalizer(iris.data)
Label = iris.target.reshape([-1,1])
DataXy = np.concatenate([Value, Label],1)
np.random.shuffle(DataXy)
X_train = DataXy[:100,:4]
y_train = DataXy[:100,-1]
X_test = DataXy[100:,:4]
y_test = DataXy[100:,-1]
data = generator([X_train, y_train], M)

### Sklearn linear

In [6]:
# sklearn linear model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_trn = regr.predict(X_train)
y_tst = regr.predict(X_test)
print('sklearn linear: \n train mse: %f\t test mse: %f' % 
      (mean_squared_error(y_train, y_trn), mean_squared_error(y_test, y_tst)))

sklearn linear: 
 train mse: 0.070231	 test mse: 0.074968


/Users/chenjunhua/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


## Model

In [7]:
# MODEL
X = tf.placeholder(tf.float32, [None, D])
y_ph = tf.placeholder(tf.float32, [None])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
y = Normal(loc=ed.dot(X, w), scale=tf.ones(1))

## Inference HIM

In [8]:
def ratio_estimator(data, local_vars, global_vars):
    """Takes as input a dict of data x, local variable samples z, and
    global variable samples beta; outputs real values of shape
    (x.shape[0] + z.shape[0],). In this example, there are no local
    variables.
    """
    # data[y] has shape (M,); global_vars[w] has shape (D,)
    # we concatenate w to each data point y, so input has shape (M, 1 + D)
    input = tf.concat([
            tf.reshape(data[y], [M, 1]),
            tf.tile(tf.reshape(global_vars[w], [1, D]), [M, 1])], 1)
    hidden = slim.fully_connected(input, 64, activation_fn=tf.nn.leaky_relu)
    output = slim.fully_connected(hidden, 1, activation_fn=None)
    return output

In [9]:
# INFERENCE

qw = Normal(loc=tf.Variable(tf.random_normal([D]) + .0),
                        scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))

inference = ed.ImplicitKLqp(
        {w: qw}, data={y: y_ph},
        discriminator=ratio_estimator, global_vars={w: qw})
inference.initialize(n_iter=5000, n_print=500)

sess = ed.get_session()
tf.global_variables_initializer().run()


for _ in range(inference.n_iter):
    X_batch, y_batch = next(data)
    for _ in range(5):
        info_dict_d = inference.update(
                variables="Disc", feed_dict={X: X_batch, y_ph: y_batch})

    info_dict = inference.update(
            variables="Gen", feed_dict={X: X_batch, y_ph: y_batch})
    info_dict['loss_d'] = info_dict_d['loss_d']
    info_dict['t'] = info_dict['t'] // 6    # say set of 6 updates is 1 iteration

    t = info_dict['t']
    inference.print_progress(info_dict)
    if t == 1 or t % inference.n_print == 0:
        # Check inferred posterior parameters.
        mean, std = sess.run([qw.mean(), qw.stddev()])
        print("\nInferred mean & std:")
        print(mean)
        print(std)
        print('#### mse: %f\ttest mse: %f\n' % 
              (ed.evaluate('mean_squared_error', data={X: X_train, y: y_train}, n_samples=500),
               ed.evaluate('mean_squared_error', data={X: X_test, y: y_test}, n_samples=500)))

   1/5000 [  0%]                                ETA: 5515s | Disc Loss: 1.027 | Gen Loss: 22.461
Inferred mean & std:
[-0.10964653 -0.21976471  0.62489611 -0.46815526]
[ 0.31523529  0.89373672  0.15893783  0.83554077]
#### mse: 1.436791	test mse: 1.800727

 500/5000 [ 10%] ███                            ETA: 43s | Disc Loss: 0.911 | Gen Loss: 22.734
Inferred mean & std:
[ 1.14194131  0.46966857  0.32792512  0.29924989]
[ 0.46790308  0.74713898  0.86422336  0.30036217]
#### mse: 1.372312	test mse: 1.839772

1000/5000 [ 20%] ██████                         ETA: 33s | Disc Loss: 1.001 | Gen Loss: 17.665
Inferred mean & std:
[-0.02411987 -0.25009912 -0.18093771 -0.66380525]
[ 0.32037061  0.69662017  0.71519917  0.29345939]
#### mse: 1.678828	test mse: 1.766991

1500/5000 [ 30%] █████████                      ETA: 28s | Disc Loss: 0.758 | Gen Loss: 18.227
Inferred mean & std:
[ 0.08030156  0.02773621  0.00449938 -0.68941444]
[ 0.31363761  0.60558254  0.71963865  0.25051531]
#### mse: 1.43862

## Inference KLqp

In [10]:
# INFERENCE v2
qw = Normal(loc=tf.Variable(tf.random_normal([D]) + .0),
                        scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))

inference = ed.KLqp({w: qw}, data={y: y_ph})
inference.initialize(n_iter=5000, n_print=500)

sess = ed.get_session()
tf.global_variables_initializer().run()


for t in range(inference.n_iter):
    X_batch, y_batch = next(data)
    info_dict = inference.update(feed_dict={X: X_batch, y_ph: y_batch})
    # info_dict = inference.update(feed_dict={X: X_train, y_ph: y_train})
    inference.print_progress(info_dict)
    if t == 1 or t % inference.n_print == 0:
        # Check inferred posterior parameters.
        mean, std = sess.run([qw.mean(), qw.stddev()])
        print("\nInferred mean & std:")
        print(mean)
        print(std)
        print('#### mse: %f\ttest mse: %f\n' % 
              (ed.evaluate('mean_squared_error', data={X: X_train, y: y_train}, n_samples=500),
               ed.evaluate('mean_squared_error', data={X: X_test, y: y_test}, n_samples=500)))

   1/5000 [  0%]                                ETA: 4733s | Loss: 218.260
Inferred mean & std:
[-0.12962687 -1.69752562  0.24212778  0.45240113]
[ 1.58082426  0.38093698  1.65014386  0.21955991]
#### mse: 1.421185	test mse: 1.723321


Inferred mean & std:
[-0.09637856 -1.70728397  0.24871224  0.41900766]
[ 1.50213301  0.36035776  1.57123733  0.23764306]
#### mse: 1.417322	test mse: 2.017613

 500/5000 [ 10%] ███                            ETA: 51s | Loss: 102.681
Inferred mean & std:
[ 0.44218385 -0.83457851  0.91158158 -0.31776103]
[ 0.10731237  0.21839711  0.19535615  0.07223265]
#### mse: 1.612435	test mse: 1.990198

1000/5000 [ 20%] ██████                         ETA: 31s | Loss: 106.589
Inferred mean & std:
[ 0.41305426 -0.84566212  0.9213053  -0.36596432]
[ 0.10845062  0.17922792  0.18315829  0.09811689]
#### mse: 1.726522	test mse: 1.875740

1500/5000 [ 30%] █████████                      ETA: 23s | Loss: 105.668
Inferred mean & std:
[ 0.3568694  -0.85863274  0.94876331 -0.3334

In [49]:
# INFERENCE v2.1
qw = Normal(loc=tf.Variable(tf.random_normal([D]) + .0),
                        scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))

inference = ed.KLqp({w: qw}, data={y: y_train, X: X_train})
inference.initialize(n_iter=5000, n_print=500)

inference.run()

print('#### mse: %f\ttest mse: %f\n' % 
      (ed.evaluate('mean_squared_error', data={X: X_train, y: y_train}, n_samples=500),
       ed.evaluate('mean_squared_error', data={X: X_test, y: y_test}, n_samples=500)))

1000/1000 [100%] ██████████████████████████████ Elapsed: 1s | Loss: 102.474
#### mse: 1.640181	test mse: 1.853891

